In [1]:
# Core
import numpy as np
import pandas as pd

# Astronomy
import rebound

In [2]:
# Local imports
import kepler_sieve
from horizons import make_sim_horizons
from rebound_sim import make_sim_planets
from rebound_integrate import integrate_numpy, integrate_df
from db_config import db_engine
from db_utils import sp2df, get_columns, df2db, df2csv, csv2db_stage, csv2db, csvs2db

In [3]:
from asteroid_element import get_ast_ref_elts, get_ast_ref_elts_jpl, add_asteroid_elts, update_asteroid_elements
from asteroid_element_synch import process_dates, calc_ref_elt

In [4]:
# Typing
from typing import Tuple, List

In [5]:
epoch_out = 59000
elt_dates = sp2df(sp_name='JPL.GetAsteroidRefElementDates', params={'epoch':epoch_out})

In [6]:
elt_dates

,epoch,AsteroidCount
0,58873,8
1,58874,19
2,58875,8
3,58876,7
4,58877,7
...,...,...
115,59011,2
116,59012,1
117,59013,2
118,59014,6


In [9]:
# First epoch
epoch0 = int(elt_dates.epoch[0])

In [25]:
# Start with a base simulation of the Sun, Planets and Moon
sim = make_sim_planets(epoch=epoch0)
# Set simulation time t to match epoch
sim.t = sim.epoch
# Set the number of active particles to the base simulation
# https://rebound.readthedocs.io/en/latest/ipython/Testparticles.html
sim.N_active = sim.N

Added bodies to simulation.


In [26]:
# Get the elements
# elts = sp2df(sp_name='KS.GetAsteroidRefElements', params={'epoch':epoch0})
elts = get_ast_ref_elts_jpl(epoch0)

In [27]:
elts

,AsteroidID,TimeID,AsteroidName,epoch,a,e,inc,Omega,omega,M
0,1413593,60416640,2060 T-2,41956.0,2.373136,0.202053,0.012784,3.08049,3.456215,6.202047


In [28]:
# First set of asteroid IDs
asteroid_ids = add_asteroid_elts(sim=sim, elts=elts)

In [29]:
asteroid_ids

array([1413593])

In [10]:
sim, elts = process_dates(epoch_out, max_dates=3)

In [11]:
elts

,AsteroidID,TimeID,AsteroidName,epoch,a,e,inc,Omega,omega,M,f
0,1413593,84960000,2060 T-2,59000.0,2.319947,0.193103,0.013013,3.085343,-2.811390,-0.435800,5.640864
1,1413594,84960000,2678 T-3,59000.0,2.243868,0.265620,0.169600,-2.723455,2.611393,-1.980524,-2.398462
2,1000007,84960000,1979 MW5,59000.0,3.185015,0.298391,0.164023,2.169598,2.773325,1.322903,-4.361898


In [32]:
max_date = 3
N_date = 1000

In [12]:
ref_elts = calc_ref_elt(sim)

In [13]:
ref_elts

,AsteroidID,TimeID,epoch,a,e,inc,Omega_node,omega_peri,f,M,d,v,h,period,mean_motion,T_peri,pomega
0,1413593.0,84960000,59000.0,2.319947,0.193103,0.013013,3.085343,3.471796,5.640864,5.847385,1.934352,0.013366,0.025725,1289.805987,0.004871,59089.460562,0.273954
1,1413594.0,84960000,59000.0,2.243868,0.265620,0.169600,3.559730,2.611393,3.884723,4.302661,2.592652,0.009825,0.024859,1226.883202,0.005121,59386.726134,6.171122
2,1000007.0,84960000,59000.0,3.185015,0.298391,0.164023,2.169598,2.773325,1.921288,1.322903,3.232631,0.009502,0.029321,2074.791898,0.003028,58563.159785,4.942923


In [17]:
schema='KS'
table='AsteroidElement_Ref'
truncate=False
chunksize=0
verbose=True
df2db(df=ref_elts, schema=schema, table=table, truncate=truncate, chunksize=chunksize, verbose=verbose)

Extracting 3 records from DataFrame into CSV files in chunks of 0 rows...
CSV file name: ../data/df2db/AsteroidElement_Ref/AsteroidElement_Ref.csv
Elapsed Time for CSV Conversion: 0:00:00.003431
Elapsed Time for DB insertion: 0:00:00.089094


In [23]:
sim.particles

AttributeError: 'Particles' object has no attribute 'a'

In [36]:
EA = 1